In [2]:
import numpy as np

In [3]:
data = np.genfromtxt('data/mushroom.train', missing_values=0, skip_header=0, delimiter=',', dtype=int)
xtrn = data[:, 1:]
ytrn = data[: , 0]

In [8]:
lena = len(xtrn)
w = np.ones((lena, 1), dtype=int)

In [9]:
w

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])